# Logistic modeling of the spread of COVID-19

In [1]:
# Import packages
import pandas as pd
import datetime as dt

import numpy as np
from scipy import optimize

from matplotlib import pyplot as plt

from tqdm import tqdm

## Preprocessing

In [2]:
# Load data
# Data retrieved from https://github.com/CSSEGISandData/COVID-19
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [3]:
# Delete province and location
cases = cases.drop(columns=['Province/State', 'Lat', 'Long'])
deaths = deaths.drop(columns=['Province/State', 'Lat', 'Long'])

In [4]:
# Group by country
cases = cases.groupby('Country/Region').sum()
deaths = deaths.groupby('Country/Region').sum()

## Regression

In [ ]:
results = pd.DataFrame(columns=['Current cases', 'Total cases', 'Inflection point cases', 'Max new cases per day',
                                'Current deaths', 'Total deaths', "Inflection point deaths", 'Death ratio (%)'], 
                       index=cases.index)

countries_to_plot = ['Germany', 'US', 'United Kingdom', 'Brazil']

def logistic_fit(w, t, y):
    return w[0] / (1 + np.exp(-w[1] * (t - w[2]))) - y
def logistic(w, t):
    return w[0] / (1 + np.exp(-w[1] * (t - w[2])))

for ind in tqdm(cases.index):
    s = cases.loc[ind]
    date = [dt.datetime.strptime(ind, '%m/%d/%y').date() for ind in s.index]
    date_int = np.array([(d - date[0]).days for d in date])
    cases_np = np.array(s)
    deaths_np = np.array(deaths.loc[ind])

    opt_cases = optimize.least_squares(logistic_fit, [100000, 0.1, 60], args=(date_int, cases_np))
    opt_deaths = optimize.least_squares(logistic_fit, [10000, 0.1, 70], args=(date_int, deaths_np))
    
    results['Current cases'].loc[ind] = cases_np[-1].astype(np.int)
    results['Current deaths'].loc[ind] = deaths_np[-1].astype(np.int)
    results['Total cases'].loc[ind] = np.round(np.max([cases_np[-1], opt_cases.x[0]])).astype(np.int)
    results['Inflection point cases'].loc[ind] = (date[0] + dt.timedelta(days=opt_cases.x[2])).strftime('%m/%d/%y')
    results['Inflection point deaths'].loc[ind] = (date[0] + dt.timedelta(days=opt_deaths.x[2])).strftime('%m/%d/%y')

    results['Max new cases per day'].loc[ind] = np.min([np.round(opt_cases.x[0] * opt_cases.x[1] / 4), 
                                                        results['Total cases'].loc[ind]]).astype(np.int)
    results['Total deaths'].loc[ind] = np.round(np.max([deaths_np[-1], np.min([np.abs(opt_deaths.x[0]), 
                                                                               results['Total cases'].loc[ind]])])).astype(np.int)
    r = np.abs(np.round(results['Total deaths'].loc[ind] / results['Total cases'].loc[ind] * 100, 2))
    results['Death ratio (%)'].loc[ind] = r if r > 1e-4 else 0
    
    if ind in countries_to_plot:
        plt.figure(figsize=(15, 10))
        plt.plot(date, cases_np, '.', markersize=12, zorder=2, label='Data')
        plt.plot(date, logistic(opt_cases.x, date_int), 'r', linewidth=2.0, zorder=1, label='Fit')
        plt.legend(loc=2, fontsize=18)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.title('Confirmed cases in ' + ind, fontsize=20)    
        x_infl = (dt.datetime.strptime(results['Inflection point cases'].loc[ind],
                                      '%m/%d/%y').date())
        plt.axvline(x=x_infl, color="green")
        plt.savefig('plots/cases_' + ind.lower() + '.png', dpi=200)
        plt.close()
        
        plt.figure(figsize=(15, 10))
        plt.plot(date, deaths_np, '.', markersize=12, zorder=2, label='Data')
        plt.plot(date, logistic(opt_deaths.x, date_int), 'r', linewidth=2.0, zorder=1, label='Fit')
        plt.legend(loc=2, fontsize=18)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.title('Deaths in ' + ind, fontsize=20)
        x_infl = (dt.datetime.strptime(results['Inflection point deaths'].loc[ind],
                                       '%m/%d/%y').date())
        plt.axvline(x=x_infl, color="green")
        plt.savefig('plots/deaths_' + ind.lower() + '.png', dpi=200)
        plt.close()

results_sorted = results.sort_values(by='Current cases', ascending=False)

  2%|▏         | 3/181 [00:00<00:17, 10.31it/s]<ipython-input-5-08165d130329>:8: RuntimeWarning: overflow encountered in exp
  return w[0] / (1 + np.exp(-w[1] * (t - w[2]))) - y
 64%|██████▎   | 115/181 [00:05<00:02, 22.03it/s]

## Save results

In [ ]:
results_sorted.to_csv(r'logistic-results.csv')